# 1 Setup & Imports

This notebook connects processed data (chunks.jsonl)
with an open-source embeddings model and Qdrant vector DB.

In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm  # Progress bar for embedding generation
from typing import List, Dict

# SentenceTransformers provides pre-trained embedding models.
from sentence_transformers import SentenceTransformer

# Qdrant client library — connects to your local or remote Qdrant instance.
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

# Paths 
DATA_DIR = "../data"
CHUNKS_FILE = os.path.join(DATA_DIR, "chunks.jsonl")


# Sanity check 
if not os.path.exists(CHUNKS_FILE):
    raise FileNotFoundError(f"Missing {CHUNKS_FILE}. Run Notebook 01 first!")

print(f"Using chunks file: {CHUNKS_FILE}")

Using chunks file: ../data/chunks.jsonl


# 2 Load Chunks from JSONL
We'll load all chunk records from the saved file.
Each chunk has: source, chunk_id, text, and type.

In [2]:
def load_chunks(file_path: str) -> List[Dict]:
    """Load a JSONL file into a Python list of dictionaries."""
    chunks = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            chunks.append(json.loads(line))
    return chunks

# --- Load and preview ---
all_chunks = load_chunks(CHUNKS_FILE)
print(f"Loaded {len(all_chunks)} text chunks.")
print("Example chunk:")
print(json.dumps(all_chunks[0], indent=2)[:100])

Loaded 1014 text chunks.
Example chunk:
{
  "id": "test_chunk0",
  "source": "../data/test.pdf",
  "chunk": "The Journey A Digital and Socie


# 3 Generate Embeddings

We'll use an open-source model from Hugging Face
(SentenceTransformers) to create vector embeddings.
These embeddings represent semantic meaning numerically.

In [3]:
# Chose an open-source embedding model
# 'all-MiniLM-L6-v2' is lightweight, free, and fast (384-dim embeddings)
model_name = "sentence-transformers/all-MiniLM-L6-v2"

print(f"Loading embedding model: {model_name}")
embedder = SentenceTransformer(model_name)

# Generate embeddings for all chunks
for chunk in tqdm(all_chunks, desc="Encoding text chunks"):
    embedding = embedder.encode(chunk["chunk"])
    chunk["embedding"] = embedding.tolist()  # Convert NumPy array → list for JSON compatibility

print("Finished generating embeddings.")

Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


Encoding text chunks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1014/1014 [00:04<00:00, 218.64it/s]

Finished generating embeddings.


# 4 Connect or Initialize Qdrant

This cell connects to a local Qdrant instance.
You can run Qdrant locally using Docker:
  docker run -p 6333:6333 qdrant/qdrant



In [4]:
# Create a Qdrant client connected to your local instance
client = QdrantClient("http://localhost:6333")

# Define collection name for our embeddings and metadata -- will change as needed
COLLECTION_NAME = "enterprise_docs"

# Create or overwrite the collection
# Qdrant organizes data into "collections", similar to tables.
# Each record will contain:
#   - vector embedding
#   - metadata (source, chunk_id, text)
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=rest.VectorParams(size=384, distance=rest.Distance.COSINE)
)

print(f"Collection '{COLLECTION_NAME}' is ready in Qdrant.")

/tmp/ipykernel_10357/1378349311.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Collection 'enterprise_docs' is ready in Qdrant.


# 5 Upload Embeddings and Metadata

We'll push each chunk’s vector and metadata to Qdrant.
This enables semantic search later on.

In [5]:
# Prepare points for Qdrant (each point = one vector + metadata)
points = []
for idx, chunk in enumerate(all_chunks):
    points.append(
        rest.PointStruct(
            id=idx,  # unique ID for this chunk
            vector=chunk["embedding"],  # the embedding vector
            payload={  # additional metadata
                "source": chunk["source"],
                "chunk_id": chunk["id"],
                "text": chunk["chunk"],
                "type": chunk.get("type", "unknown")
            }
        )
    )

# --- Upload to Qdrant ---
client.upsert(collection_name=COLLECTION_NAME, points=points)

print(f"Uploaded {len(points)} chunks to collection '{COLLECTION_NAME}'.")

Uploaded 1014 chunks to collection 'enterprise_docs'.


# 6 Test Semantic Search

Let's try asking a question, embed it, and search similar content
from our stored chunks in Qdrant.

In [6]:
# Example user query
query = "What does 6G offer?"

# Generate embedding for the query
query_vector = embedder.encode(query).tolist()

# Perform vector similarity search in Qdrant
search_results = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    limit=3  # Top 3 most relevant chunks
)

# --- Display results ---
print(f"\n🔍 Query: {query}\n")
for i, hit in enumerate(search_results):
    print(f"Result {i+1} — Score: {hit.score:.4f}")
    print(f"Source: {hit.payload['source']}")
    print(f"Text snippet: {hit.payload['text'][:]}...")
    print("-" * 80)


🔍 Query: What does 6G offer?

Result 1 — Score: 0.4660
Source: ../data/test.pdf
Text snippet: The Journey A Digital and Societal Lina Mohjazi, Bassant Selim, Mallik Abstract—While the fifth generation (5 G) is bringing an innovative fabric of breakthrough technologies, enabling smart factories, cities, and Internet-of-Things (Io T), the unprecedented strain on communication networks put by these applications, in terms of highly cognitive, agile architectures and the support of massive connectivity, energy efficiency, and extreme ultralow latency, is pushing 5 G to their limits. As such, the focus of academic and industrial efforts has shifted toward beyond 5 G (B 5 G) and the conceptualization of sixth generation (6 G) systems. This article discusses four main digital and societal use cases (UCs) that will drive the need to reconcile a new breed of network requirements. Based on this, we provide our vision of the fundamental architectural ingredients that will enable the promise of 6 

/tmp/ipykernel_10357/819005533.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(
